In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
!pip install Augmentor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers import Input, Lambda, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/FYP/DataSet/Trainning'
valid_path = '/content/drive/MyDrive/FYP/DataSet/Validation'
test_path = '/content/drive/MyDrive/FYP/DataSet/Test'


In [ ]:
# re-load the VGG19 model with imagenet pre-trained weights
vgg = VGG19(input_shape=(224,224,3), weights='imagenet', include_top=False)

# freeze the layers
for layer in vgg.layers:
    layer.trainable = False

# add new layers
x = Flatten()(vgg.output)
x = Dense(units=4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=2, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=x)

# view the structure of the model
model.summary()

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#Using ImageDataGenerator to read images from directories

#Rescale all images by 1/255

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

testing_set = valid_datagen.flow_from_directory(test_path,
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical')

In [ ]:
from os import listdir

label = listdir(train_path)
numClass = len(label)
print (label)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

 #checkpoint
checkpoint = ModelCheckpoint("/content/drive/MyDrive/FYP/modelV2_VGG19_8to2_ep20.h5",verbose= 1, save_best_only = True)

# # don't train existing weights
# for layer in vgg.layers:
#     layer.trainable = False

r = model.fit(training_set, epochs=20, validation_data=testing_set, validation_steps=1,
          shuffle=True,
          callbacks=checkpoint)

In [ ]:
from imutils import paths 
import os
from os import listdir
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools


In [ ]:
test_imgs, test_labels = next(testing_set)
# ensemble predictions and submit
results = np.zeros( (200,2) )
results = model.predict(test_imgs, verbose = 1)
results = np.argmax(results, axis = -1)
print(results)
test_labels = np.argmax(test_labels,axis = -1)
test_labels
print(test_labels)
# confusion matrix
cm = confusion_matrix(y_true=test_labels, y_pred=results)
cm = pd.DataFrame(cm , index = ['0','1'] , columns = ['0','1'])
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "OrRd", linecolor = 'white' , linewidth = 1 , annot = True, fmt='', xticklabels = label, yticklabels = label)
plt.show()


In [ ]:
test_predict = model.predict(testing_set)

predict_class1 = np.argmax(test_predict, axis=1)
predict_class1 = predict_class1.tolist()

labels = ['abnormal', 'normal']
report1 = classification_report(testing_set.classes,predict_class1, target_names=labels)
print(report1)

In [ ]:
from imutils import paths 
import os
from os import listdir
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# load the best model
from keras.models import load_model
model = load_model("/content/drive/MyDrive/FYP/modelV2_VGG19_8to2_ep20.h5")

classname = ["abnormal", "normal"]
data = []
labels = []
imagePaths = sorted(list(paths.list_images(test_path)))

for imgPath in imagePaths:
    img = cv2.imread(imgPath)
    ori = img.copy()
    img = cv2.resize(img, (224, 224))
    img = img_to_array(img)
    lbl = imgPath.split(os.path.sep)[-1]
    data.append(img)
    labels.append(lbl)

data = np.array(data)/255
labels = np.array(labels)
prediction = model.predict(data)
y_pred = np.argmax(prediction, axis=1)

for i in range(len(imagePaths)):
    a = prediction[i]
    ind = np.argmax(a)
    print(labels[i], " ", ind, classname[ind])

print(y_pred)
print('Classification report')
print(classification_report(testing_set.classes, y_pred, target_names=classname))
print('Confusion Matrix')
print(confusion_matrix(testing_set.classes, y_pred))

cm = confusion_matrix(testing_set.classes, y_pred)
plt.figure(figsize=(12, 12))
plt.title("Confusion Matrix")
ax = sns.heatmap(cm, cmap=plt.cm.Greens, annot=True, fmt=".0f", xticklabels=classname, yticklabels=classname)
plt.rcParams.update({"font.size":30})
ax.tick_params(axis="both", which="major", labelsize=20)
ax.set_ylabel("Actual", fontsize=36)
ax.set_xlabel("Predicted", fontsize=36)
plt.show()
